# Week 04: Recursion & Binary Tree 기초 - 실습 노트북

---

## 🔧 필수 라이브러리 Import

In [1]:
# 표준 라이브러리
import sys
from typing import List, Optional, Tuple, Dict, Set
import time

# 자료구조 관련
from collections import deque, defaultdict, Counter
from heapq import heappush, heappop, heapify
from bisect import bisect_left, bisect_right

# 알고리즘 관련
from functools import lru_cache, reduce
from itertools import permutations, combinations, product, accumulate
import math

# 재귀 깊이 설정
sys.setrecursionlimit(10**6)

print("Python version:", sys.version)
print("재귀 깊이 제한:", sys.getrecursionlimit())
print("라이브러리 import 완료!")

Python version: 3.13.7 (main, Aug 14 2025, 11:12:11) [Clang 17.0.0 (clang-1700.0.13.3)]
재귀 깊이 제한: 1000000
라이브러리 import 완료!


## 🎯 Section 1: 재귀(Recursion) 기초부터 심화까지

### 1.1 재귀의 기본 개념과 성능 비교

In [ ]:
# 재귀의 3가지 핵심 요소
# 1. Base Case (종료 조건)
# 2. Recursive Case (재귀 호출)
# 3. Progress toward Base Case (종료 조건에 가까워짐)

def factorial_recursive(n):
    """
    재귀로 구현한 팩토리얼
    시간복잡도: O(n)
    공간복잡도: O(n) - 콜 스택
    """
    if n <= 1:  # Base case
        return 1
    return n * factorial_recursive(n - 1)  # Recursive case

def factorial_iterative(n):
    """
    반복문으로 구현한 팩토리얼
    시간복잡도: O(n)
    공간복잡도: O(1) - 더 효율적!
    """
    result = 1
    for i in range(2, n + 1):
        result *= i
    return result

# 성능 비교
print("=== 팩토리얼 테스트 ===")
for n in [5, 10, 20]:
    # 재귀
    start = time.perf_counter()
    result_rec = factorial_recursive(n)
    time_rec = time.perf_counter() - start
    
    # 반복문
    start = time.perf_counter()
    result_iter = factorial_iterative(n)
    time_iter = time.perf_counter() - start
    
    print(f"n={n}: {result_rec}")
    print(f"  재귀: {time_rec*1000:.4f}ms")
    print(f"  반복문: {time_iter*1000:.4f}ms")
    print(f"  반복문이 {time_rec/time_iter:.1f}배 빠름\n")

In [ ]:
# 피보나치: 재귀의 문제점과 해결책
def fibonacci_naive(n):
    """
    순진한 재귀 피보나치 - 매우 비효율적!
    시간복잡도: O(2^n) - 지수적 증가!
    공간복잡도: O(n)
    
    문제: 중복 계산
    fib(5) = fib(4) + fib(3)
           = (fib(3) + fib(2)) + fib(3)  # fib(3)이 2번 계산됨
    """
    if n <= 1:
        return n
    return fibonacci_naive(n-1) + fibonacci_naive(n-2)

@lru_cache(maxsize=None)
def fibonacci_memo(n):
    """
    메모이제이션을 적용한 피보나치
    시간복잡도: O(n) - 각 값을 한 번만 계산
    공간복잡도: O(n) - 캐시 저장
    """
    if n <= 1:
        return n
    return fibonacci_memo(n-1) + fibonacci_memo(n-2)

def fibonacci_iterative(n):
    """
    반복문 피보나치 - 가장 효율적
    시간복잡도: O(n)
    공간복잡도: O(1) - 상수 공간만 사용
    """
    if n <= 1:
        return n
    
    prev, curr = 0, 1
    for _ in range(2, n + 1):
        prev, curr = curr, prev + curr
    return curr

print("=== 피보나치 성능 비교 ===")
for n in [10, 20, 30]:
    print(f"n={n}:")
    
    # 순진한 재귀 (n이 작을 때만)
    if n <= 30:
        start = time.perf_counter()
        result_naive = fibonacci_naive(n)
        time_naive = time.perf_counter() - start
        print(f"  순진한 재귀: {result_naive} ({time_naive*1000:.2f}ms)")
    
    # 메모이제이션
    fibonacci_memo.cache_clear()  # 캐시 초기화
    start = time.perf_counter()
    result_memo = fibonacci_memo(n)
    time_memo = time.perf_counter() - start
    print(f"  메모이제이션: {result_memo} ({time_memo*1000:.4f}ms)")
    
    # 반복문
    start = time.perf_counter()
    result_iter = fibonacci_iterative(n)
    time_iter = time.perf_counter() - start
    print(f"  반복문: {result_iter} ({time_iter*1000:.4f}ms)")
    
    # 캐시 정보
    cache_info = fibonacci_memo.cache_info()
    print(f"  캐시 통계: hits={cache_info.hits}, misses={cache_info.misses}")
    print()

In [ ]:
## 💡 Section 2: Divide and Conquer 고급 분석

### 2.1 Master Theorem과 거듭제곱 최적화

## 💡 Section 2: Divide and Conquer (분할 정복)

### 2.1 거듭제곱 계산 - O(log n) 최적화

In [5]:
# 거듭제곱: 단순 vs 분할 정복
def power_naive(x, n):
    """
    단순 거듭제곱
    시간복잡도: O(n)
    """
    if n == 0:
        return 1
    
    result = 1
    for _ in range(abs(n)):
        result *= x
    
    return result if n > 0 else 1/result

def power_divide_conquer(x, n):
    """
    분할 정복 거듭제곱
    시간복잡도: O(log n)
    """
    if n == 0:
        return 1
    if n < 0:
        return 1 / power_divide_conquer(x, -n)
    
    # 짝수인 경우: x^n = (x^(n/2))^2
    if n % 2 == 0:
        half = power_divide_conquer(x, n // 2)
        return half * half
    # 홀수인 경우: x^n = x * x^(n-1)
    else:
        return x * power_divide_conquer(x, n - 1)

print("=== 거듭제곱 성능 비교 ===")
test_cases = [(2, 10), (3, 15), (2, 20)]

for x, n in test_cases:
    # 단순 방법
    start = time.perf_counter()
    result_naive = power_naive(x, n)
    time_naive = time.perf_counter() - start
    
    # 분할 정복
    start = time.perf_counter()
    result_dc = power_divide_conquer(x, n)
    time_dc = time.perf_counter() - start
    
    print(f"{x}^{n} = {result_dc}")
    print(f"  단순: {time_naive*1000:.4f}ms")
    print(f"  분할정복: {time_dc*1000:.4f}ms")
    print(f"  속도 향상: {time_naive/time_dc:.1f}배\n")

=== 거듭제곱 성능 비교 ===
2^10 = 1024
  단순: 0.0015ms
  분할정복: 0.0016ms
  속도 향상: 0.9배

3^15 = 14348907
  단순: 0.0010ms
  분할정복: 0.0011ms
  속도 향상: 0.9배

2^20 = 1048576
  단순: 0.0007ms
  분할정복: 0.0007ms
  속도 향상: 1.0배



### 2.2 병합 정렬 - 재귀적 분할 정복

In [6]:
def merge_sort(arr):
    """
    병합 정렬 - 분할 정복의 대표적 예
    시간복잡도: O(n log n)
    공간복잡도: O(n)
    """
    # 베이스 케이스
    if len(arr) <= 1:
        return arr
    
    # 분할 (Divide)
    mid = len(arr) // 2
    left = merge_sort(arr[:mid])
    right = merge_sort(arr[mid:])
    
    # 정복 및 결합 (Conquer & Combine)
    return merge(left, right)

def merge(left, right):
    """두 정렬된 배열을 병합"""
    result = []
    i = j = 0
    
    while i < len(left) and j < len(right):
        if left[i] <= right[j]:
            result.append(left[i])
            i += 1
        else:
            result.append(right[j])
            j += 1
    
    # 남은 원소 처리
    result.extend(left[i:])
    result.extend(right[j:])
    return result

# 테스트
print("=== 병합 정렬 테스트 ===")
test_arrays = [
    [64, 34, 25, 12, 22, 11, 90],
    [5, 2, 8, 1, 9],
    [1],
    [3, 3, 3, 3]
]

for arr in test_arrays:
    original = arr.copy()
    sorted_arr = merge_sort(arr)
    print(f"원본: {original}")
    print(f"정렬: {sorted_arr}")
    print(f"올바른 정렬: {sorted_arr == sorted(original)}\n")

=== 병합 정렬 테스트 ===
원본: [64, 34, 25, 12, 22, 11, 90]
정렬: [11, 12, 22, 25, 34, 64, 90]
올바른 정렬: True

원본: [5, 2, 8, 1, 9]
정렬: [1, 2, 5, 8, 9]
올바른 정렬: True

원본: [1]
정렬: [1]
올바른 정렬: True

원본: [3, 3, 3, 3]
정렬: [3, 3, 3, 3]
올바른 정렬: True



## 🌳 Section 3: Binary Tree 구현과 순회

### 3.1 TreeNode 클래스와 트리 생성

In [7]:
class TreeNode:
    """이진 트리 노드 클래스"""
    def __init__(self, val=0, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right
    
    def __repr__(self):
        return f"TreeNode({self.val})"

def build_tree(values):
    """리스트로부터 이진 트리 생성 (레벨 순서)"""
    if not values or values[0] is None:
        return None
    
    root = TreeNode(values[0])
    queue = deque([root])
    i = 1
    
    while queue and i < len(values):
        node = queue.popleft()
        
        # 왼쪽 자식
        if i < len(values) and values[i] is not None:
            node.left = TreeNode(values[i])
            queue.append(node.left)
        i += 1
        
        # 오른쪽 자식
        if i < len(values) and values[i] is not None:
            node.right = TreeNode(values[i])
            queue.append(node.right)
        i += 1
    
    return root

def print_tree(root, level=0, prefix="Root: "):
    """트리를 시각적으로 출력"""
    if root:
        print(" " * (level * 4) + prefix + str(root.val))
        if root.left or root.right:
            if root.left:
                print_tree(root.left, level + 1, "L--- ")
            else:
                print(" " * ((level + 1) * 4) + "L--- None")
            if root.right:
                print_tree(root.right, level + 1, "R--- ")
            else:
                print(" " * ((level + 1) * 4) + "R--- None")

# 예제 트리 생성
#       1
#      / \
#     2   3
#    / \
#   4   5
print("=== 이진 트리 생성 ===")
tree_values = [1, 2, 3, 4, 5, None, None]
root = build_tree(tree_values)
print("트리 구조:")
print_tree(root)

=== 이진 트리 생성 ===
트리 구조:
Root: 1
    L--- 2
        L--- 4
        R--- 5
    R--- 3


### 3.2 네 가지 트리 순회 방법

In [8]:
# 1. 전위 순회 (Preorder): 루트 → 왼쪽 → 오른쪽
def preorder_recursive(root):
    """전위 순회 - 재귀"""
    if not root:
        return []
    
    result = [root.val]
    result.extend(preorder_recursive(root.left))
    result.extend(preorder_recursive(root.right))
    return result

def preorder_iterative(root):
    """전위 순회 - 반복문 (스택)"""
    if not root:
        return []
    
    result = []
    stack = [root]
    
    while stack:
        node = stack.pop()
        result.append(node.val)
        
        # 오른쪽을 먼저 넣어야 왼쪽이 먼저 처리됨
        if node.right:
            stack.append(node.right)
        if node.left:
            stack.append(node.left)
    
    return result

# 2. 중위 순회 (Inorder): 왼쪽 → 루트 → 오른쪽
def inorder_recursive(root):
    """중위 순회 - 재귀"""
    if not root:
        return []
    
    result = []
    result.extend(inorder_recursive(root.left))
    result.append(root.val)
    result.extend(inorder_recursive(root.right))
    return result

def inorder_iterative(root):
    """중위 순회 - 반복문 (스택)"""
    result = []
    stack = []
    current = root
    
    while stack or current:
        # 왼쪽 끝까지 이동
        while current:
            stack.append(current)
            current = current.left
        
        # 노드 처리
        current = stack.pop()
        result.append(current.val)
        
        # 오른쪽 서브트리로 이동
        current = current.right
    
    return result

# 3. 후위 순회 (Postorder): 왼쪽 → 오른쪽 → 루트
def postorder_recursive(root):
    """후위 순회 - 재귀"""
    if not root:
        return []
    
    result = []
    result.extend(postorder_recursive(root.left))
    result.extend(postorder_recursive(root.right))
    result.append(root.val)
    return result

# 4. 레벨 순회 (Level-order): BFS
def level_order(root):
    """레벨 순회 - 큐 사용"""
    if not root:
        return []
    
    result = []
    queue = deque([root])
    
    while queue:
        level_size = len(queue)
        level = []
        
        for _ in range(level_size):
            node = queue.popleft()
            level.append(node.val)
            
            if node.left:
                queue.append(node.left)
            if node.right:
                queue.append(node.right)
        
        result.append(level)
    
    return result

# 테스트
print("=== 트리 순회 결과 ===")
print("트리 구조: [1, 2, 3, 4, 5]")
print("       1")
print("      / \\")
print("     2   3")
print("    / \\")
print("   4   5\n")

tree = build_tree([1, 2, 3, 4, 5])

print(f"전위 순회 (재귀): {preorder_recursive(tree)}")
print(f"전위 순회 (반복): {preorder_iterative(tree)}")
print(f"중위 순회 (재귀): {inorder_recursive(tree)}")
print(f"중위 순회 (반복): {inorder_iterative(tree)}")
print(f"후위 순회 (재귀): {postorder_recursive(tree)}")
print(f"레벨 순회: {level_order(tree)}")

=== 트리 순회 결과 ===
트리 구조: [1, 2, 3, 4, 5]
       1
      / \
     2   3
    / \
   4   5

전위 순회 (재귀): [1, 2, 4, 5, 3]
전위 순회 (반복): [1, 2, 4, 5, 3]
중위 순회 (재귀): [4, 2, 5, 1, 3]
중위 순회 (반복): [4, 2, 5, 1, 3]
후위 순회 (재귀): [4, 5, 2, 3, 1]
레벨 순회: [[1], [2, 3], [4, 5]]


## 🎨 Section 4: 백트래킹(Backtracking) 입문

### 4.1 Generate Parentheses - 백트래킹의 기본

In [9]:
def generate_parentheses(n):
    """
    LeetCode 22: Generate Parentheses
    n쌍의 올바른 괄호 조합 생성
    
    백트래킹 접근법:
    1. 여는 괄호는 n개까지 추가 가능
    2. 닫는 괄호는 여는 괄호보다 적을 때만 추가 가능
    3. 총 2n개가 되면 완성
    """
    result = []
    
    def backtrack(current, open_count, close_count):
        # 베이스 케이스: 완성된 조합
        if len(current) == 2 * n:
            result.append(current)
            return
        
        # 여는 괄호 추가 가능
        if open_count < n:
            backtrack(current + '(', open_count + 1, close_count)
        
        # 닫는 괄호 추가 가능
        if close_count < open_count:
            backtrack(current + ')', open_count, close_count + 1)
    
    backtrack('', 0, 0)
    return result

# 상태 공간 트리 시각화
def visualize_parentheses_tree(n, max_depth=4):
    """백트래킹 과정을 시각화"""
    print(f"=== n={n}의 상태 공간 트리 (깊이 {max_depth}까지) ===")
    
    def print_state(current, open_count, close_count, depth=0):
        if depth > max_depth:
            return
        
        indent = "  " * depth
        state = current if current else "시작"
        print(f"{indent}{state} (열림:{open_count}, 닫힘:{close_count})")
        
        # 여는 괄호 추가 가능
        if open_count < n:
            print_state(current + '(', open_count + 1, close_count, depth + 1)
        
        # 닫는 괄호 추가 가능
        if close_count < open_count:
            print_state(current + ')', open_count, close_count + 1, depth + 1)
    
    print_state('', 0, 0)

# 테스트
print("=== Generate Parentheses 테스트 ===")
for n in range(1, 4):
    result = generate_parentheses(n)
    print(f"n={n}: {len(result)}개의 조합")
    print(f"  {result}")
    print()

# 상태 공간 트리 시각화
visualize_parentheses_tree(2, max_depth=6)

=== Generate Parentheses 테스트 ===
n=1: 1개의 조합
  ['()']

n=2: 2개의 조합
  ['(())', '()()']

n=3: 5개의 조합
  ['((()))', '(()())', '(())()', '()(())', '()()()']

=== n=2의 상태 공간 트리 (깊이 6까지) ===
시작 (열림:0, 닫힘:0)
  ( (열림:1, 닫힘:0)
    (( (열림:2, 닫힘:0)
      (() (열림:2, 닫힘:1)
        (()) (열림:2, 닫힘:2)
    () (열림:1, 닫힘:1)
      ()( (열림:2, 닫힘:1)
        ()() (열림:2, 닫힘:2)


## 🔥 Section 5: LeetCode 문제 풀이

### Problem 1: [100] Same Tree

In [10]:
"""
문제: 두 이진 트리가 동일한지 확인
- 구조와 노드 값이 모두 같아야 함

예시:
Input: p = [1,2,3], q = [1,2,3]
Output: true

Input: p = [1,2], q = [1,null,2]
Output: false
"""

class Solution:
    def isSameTree(self, p: Optional[TreeNode], q: Optional[TreeNode]) -> bool:
        """
        재귀 접근법:
        1. 둘 다 None이면 True
        2. 하나만 None이면 False
        3. 값이 다르면 False
        4. 왼쪽과 오른쪽 서브트리 재귀 확인
        
        시간복잡도: O(min(m, n)) - m, n은 각 트리의 노드 수
        공간복잡도: O(min(m, n)) - 재귀 스택
        """
        # 베이스 케이스
        if not p and not q:
            return True
        if not p or not q:
            return False
        if p.val != q.val:
            return False
        
        # 재귀 호출
        return (self.isSameTree(p.left, q.left) and 
                self.isSameTree(p.right, q.right))

# 테스트
solution = Solution()
test_cases = [
    ([1, 2, 3], [1, 2, 3], True),
    ([1, 2], [1, None, 2], False),
    ([1, 2, 1], [1, 1, 2], False),
    ([], [], True),
    ([1], [1], True)
]

print("=== Same Tree 테스트 ===")
for i, (tree1_vals, tree2_vals, expected) in enumerate(test_cases, 1):
    tree1 = build_tree(tree1_vals) if tree1_vals else None
    tree2 = build_tree(tree2_vals) if tree2_vals else None
    result = solution.isSameTree(tree1, tree2)
    print(f"테스트 {i}: {'✅' if result == expected else '❌'}")
    print(f"  트리1: {tree1_vals}")
    print(f"  트리2: {tree2_vals}")
    print(f"  결과: {result}, 예상: {expected}\n")

=== Same Tree 테스트 ===
테스트 1: ✅
  트리1: [1, 2, 3]
  트리2: [1, 2, 3]
  결과: True, 예상: True

테스트 2: ✅
  트리1: [1, 2]
  트리2: [1, None, 2]
  결과: False, 예상: False

테스트 3: ✅
  트리1: [1, 2, 1]
  트리2: [1, 1, 2]
  결과: False, 예상: False

테스트 4: ✅
  트리1: []
  트리2: []
  결과: True, 예상: True

테스트 5: ✅
  트리1: [1]
  트리2: [1]
  결과: True, 예상: True



### Problem 2: [104] Maximum Depth of Binary Tree

In [11]:
"""
문제: 이진 트리의 최대 깊이 구하기
- 루트에서 가장 먼 리프까지의 노드 개수

예시:
Input: root = [3,9,20,null,null,15,7]
Output: 3
"""

class Solution:
    def maxDepth_recursive(self, root: Optional[TreeNode]) -> int:
        """
        재귀 접근법 (DFS):
        1. None이면 0 반환
        2. 왼쪽과 오른쪽 서브트리의 깊이 중 큰 값 + 1
        
        시간복잡도: O(n)
        공간복잡도: O(h) - h는 트리 높이
        """
        if not root:
            return 0
        
        left_depth = self.maxDepth_recursive(root.left)
        right_depth = self.maxDepth_recursive(root.right)
        
        return max(left_depth, right_depth) + 1
    
    def maxDepth_iterative(self, root: Optional[TreeNode]) -> int:
        """
        반복문 접근법 (BFS):
        레벨 순회하며 레벨 수 계산
        
        시간복잡도: O(n)
        공간복잡도: O(w) - w는 트리의 최대 너비
        """
        if not root:
            return 0
        
        depth = 0
        queue = deque([root])
        
        while queue:
            depth += 1
            level_size = len(queue)
            
            for _ in range(level_size):
                node = queue.popleft()
                if node.left:
                    queue.append(node.left)
                if node.right:
                    queue.append(node.right)
        
        return depth

# 테스트
solution = Solution()
test_cases = [
    ([3, 9, 20, None, None, 15, 7], 3),
    ([1, None, 2], 2),
    ([1], 1),
    ([], 0),
    ([1, 2, 3, 4, None, None, 5, 6], 4)
]

print("=== Maximum Depth 테스트 ===")
for i, (tree_vals, expected) in enumerate(test_cases, 1):
    tree = build_tree(tree_vals) if tree_vals else None
    result_rec = solution.maxDepth_recursive(tree)
    result_iter = solution.maxDepth_iterative(tree)
    
    print(f"테스트 {i}: {'✅' if result_rec == expected else '❌'}")
    print(f"  트리: {tree_vals}")
    print(f"  재귀 결과: {result_rec}")
    print(f"  반복 결과: {result_iter}")
    print(f"  예상: {expected}\n")

=== Maximum Depth 테스트 ===
테스트 1: ✅
  트리: [3, 9, 20, None, None, 15, 7]
  재귀 결과: 3
  반복 결과: 3
  예상: 3

테스트 2: ✅
  트리: [1, None, 2]
  재귀 결과: 2
  반복 결과: 2
  예상: 2

테스트 3: ✅
  트리: [1]
  재귀 결과: 1
  반복 결과: 1
  예상: 1

테스트 4: ✅
  트리: []
  재귀 결과: 0
  반복 결과: 0
  예상: 0

테스트 5: ✅
  트리: [1, 2, 3, 4, None, None, 5, 6]
  재귀 결과: 4
  반복 결과: 4
  예상: 4



### Problem 3: [101] Symmetric Tree

In [12]:
"""
문제: 이진 트리가 대칭(거울상)인지 확인

예시:
Input: root = [1,2,2,3,4,4,3]
Output: true

    1
   / \
  2   2
 / \ / \
3  4 4  3
"""

class Solution:
    def isSymmetric(self, root: Optional[TreeNode]) -> bool:
        """
        재귀 접근법:
        두 서브트리가 거울상인지 확인
        
        시간복잡도: O(n)
        공간복잡도: O(h)
        """
        if not root:
            return True
        
        def isMirror(left: Optional[TreeNode], right: Optional[TreeNode]) -> bool:
            # 둘 다 None
            if not left and not right:
                return True
            # 하나만 None
            if not left or not right:
                return False
            # 값이 다름
            if left.val != right.val:
                return False
            
            # 거울상 확인: left.left와 right.right, left.right와 right.left
            return (isMirror(left.left, right.right) and 
                    isMirror(left.right, right.left))
        
        return isMirror(root.left, root.right)

# 테스트
solution = Solution()
test_cases = [
    ([1, 2, 2, 3, 4, 4, 3], True),
    ([1, 2, 2, None, 3, None, 3], False),
    ([1], True),
    ([1, 2, 2, 2, None, 2], False)
]

print("=== Symmetric Tree 테스트 ===")
for i, (tree_vals, expected) in enumerate(test_cases, 1):
    tree = build_tree(tree_vals) if tree_vals else None
    result = solution.isSymmetric(tree)
    
    print(f"테스트 {i}: {'✅' if result == expected else '❌'}")
    print(f"  트리: {tree_vals}")
    print(f"  결과: {result}, 예상: {expected}\n")

=== Symmetric Tree 테스트 ===
테스트 1: ✅
  트리: [1, 2, 2, 3, 4, 4, 3]
  결과: True, 예상: True

테스트 2: ✅
  트리: [1, 2, 2, None, 3, None, 3]
  결과: False, 예상: False

테스트 3: ✅
  트리: [1]
  결과: True, 예상: True

테스트 4: ✅
  트리: [1, 2, 2, 2, None, 2]
  결과: False, 예상: False



<>:11: SyntaxWarning: invalid escape sequence '\ '
<>:11: SyntaxWarning: invalid escape sequence '\ '
/var/folders/gq/6d86vn156bld3dzv3wv1spqh0000gn/T/ipykernel_12022/2352005979.py:11: SyntaxWarning: invalid escape sequence '\ '
  / \ / \


## 📝 Section 6: 핵심 정리 및 팁

### 이번 주 핵심 포인트

#### 1. **재귀(Recursion)의 핵심**
- 베이스 케이스를 먼저 정의하라
- 문제를 더 작은 부분 문제로 분할할 수 있는지 확인
- 콜 스택을 이해하고 공간 복잡도 고려
- 필요시 Helper 함수 활용

#### 2. **Binary Tree 순회의 핵심**
- DFS (전위, 중위, 후위): 재귀 또는 스택
- BFS (레벨 순회): 큐 사용
- 트리 문제는 대부분 재귀로 우아하게 해결 가능
- Bottom-up vs Top-down 접근법 구분

#### 3. **백트래킹의 핵심**
- 상태 공간 트리를 그려보라
- 가지치기(Pruning)로 불필요한 탐색 제거
- 상태 복원(백트래킹)을 잊지 말 것

#### 4. **자주 하는 실수**
- ❌ 베이스 케이스 누락 → 무한 재귀
- ❌ None 체크 누락 → AttributeError
- ❌ 재귀 깊이 제한 미설정 → RecursionError
- ✅ 항상 엣지 케이스(빈 트리, 단일 노드) 테스트

#### 5. **Python 재귀 최적화 팁**
```python
# 재귀 깊이 늘리기
import sys
sys.setrecursionlimit(10**6)

# 메모이제이션 (Week 5에서 자세히)
from functools import lru_cache

@lru_cache(maxsize=None)
def recursive_function(n):
    # 중복 계산 방지
    pass

# Tail Recursion은 Python에서 최적화되지 않음
# 필요시 반복문으로 변환 고려
```